## Import library

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.optimizers as opt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 2"  

In [2]:
def get_available_gpus():
    """
    code from http://stackoverflow.com/questions/38559755/how-to-get-current-available-gpus-in-tensorflow
    """
    from tensorflow.python.client import device_lib as _device_lib
    local_device_protos = _device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

In [3]:
get_available_gpus()

['/device:CPU:0', '/device:GPU:0']

## Define path & total number

In [4]:
path = "C:/users/cs623/.keras/datasets/personal_cats_and_dogs"
path_cat = os.path.join(path, 'cats')
path_dog = os.path.join(path, 'dogs')

In [5]:
num_cats = len(os.listdir(path_cat))
num_dogs = len(os.listdir(path_dog))
total_img_num = num_cats + num_dogs
print('total cat images:', num_cats)
print('total dog images:', num_dogs)
print('total images:', total_img_num)

total cat images: 1500
total dog images: 2172
total images: 3672


## Train test split

In [6]:
batch_size = 4
epochs = 30
IMG_HEIGHT = 100
IMG_WIDTH = 100
split = 0.2
total_val = total_img_num * split
total_train = total_img_num - total_val

In [7]:
# Generator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=split)

#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,

In [8]:
# Load images from the disk, applies rescaling, and resizes the images

train_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='validation') # set as validation data

Found 2938 images belonging to 2 classes.
Found 734 images belonging to 2 classes.


## Labels

In [9]:
import collections

def count_label(generator):
    l = len(generator)
    counter = collections.Counter()
    for i in range(l):
        _, label = generator[i]
        unique, counts = np.unique(label, return_counts=True)
        count_label = dict(zip(unique, counts))
        counter.update(count_label) 
    result = dict(counter)
    return result

In [10]:
# train_count = count_label(train_generator)
# val_count = count_label(validation_generator)

# print("Training labels:")
# print(train_count)
# print("Validation labels:")
# print(val_count)

## Sample visualisation

In [11]:
sample_img_train, sample_label_train = next(train_generator)

In [12]:
def plotImg(img):
    fig, axes = plt.subplots(3, 5, figsize=(20, 20))
    axes = axes.flatten()
    for i, a in zip(img, axes):
        a.imshow(i)
        a.axis("off")
    plt.tight_layout()
    plt.show()

In [13]:
# plotImg(sample_img_train[:15])

## Model

In [14]:
VGGNet = Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(128, 3, padding='same', activation='relu'),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax'),
    Dense(1, activation='sigmoid'),
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(32, 3, padding='same', activation='relu'),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Conv2D(64, 3, padding='same', activation='relu'),
    Conv2D(64, 3, padding='same', activation='relu'),
#     Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    # Conv2D(64, 3, padding='same', activation='relu'),
    # Conv2D(64, 3, padding='same', activation='relu'),
    # Conv2D(64, 3, padding='same', activation='relu'),
    # MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [16]:
# from tensorflow.keras.callbacks import LearningRateScheduler
# lr_base = 0.001
# epochs = 250
# lr_power = 0.9
# def lr_scheduler(epoch, mode='power_decay'):
#     '''if lr_dict.has_key(epoch):
#         lr = lr_dict[epoch]
#         print 'lr: %f' % lr'''
 
#     if mode is 'power_decay':
#         # original lr scheduler
#         lr = lr_base * ((1 - float(epoch) / epochs) ** lr_power)
#     if mode is 'exp_decay':
#         # exponential decay
#         lr = (float(lr_base) ** float(lr_power)) ** float(epoch + 1)
#     # adam default lr
#     if mode is 'adam':
#         lr = 0.001
 
#     if mode is 'progressive_drops':
#         # drops as progression proceeds, good for sgd
#         if epoch > 0.9 * epochs:
#             lr = 0.0001
#         elif epoch > 0.75 * epochs:
#             lr = 0.001
#         elif epoch > 0.5 * epochs:
#             lr = 0.01
#         else:
#             lr = 0.1
 
#     print('lr: %f' % lr)
#     return lr
 
# # 学习率调度器
# scheduler = LearningRateScheduler(lr_scheduler)

In [17]:
sgd = opt.SGD(lr=1e-1, decay=1e-6, momentum=0.8, nesterov=True)

In [18]:
VGGNet.compile(optimizer=sgd,
             loss='mean_squared_error',
             metrics=['accuracy'],
             )

In [19]:
VGGNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 256)       2

## Train model

In [20]:
# import tensorflow as tf
# import keras.backend.tensorflow_backend as KTF

# config = tf.ConfigProto()  
# config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
# session = tf.Session(config=config)

# # 设置session
# KTF.set_session(sess)

In [21]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  
with tf.device('/cpu:0'):
    history = VGGNet.fit_generator(
        train_generator,
        steps_per_epoch = total_train // batch_size,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = total_val // batch_size     
    )

Epoch 1/30
734/734 [==============================] - 107s 146ms/step - loss: 0.2442 - acc: 0.5917 - val_loss: 0.2426 - val_acc: 0.5902
Epoch 2/30
734/734 [==============================] - 101s 138ms/step - loss: 0.2435 - acc: 0.5832 - val_loss: 0.2539 - val_acc: 0.5902
Epoch 3/30
734/734 [==============================] - 100s 137ms/step - loss: 0.2446 - acc: 0.5825 - val_loss: 0.2453 - val_acc: 0.5902
Epoch 4/30
734/734 [==============================] - 101s 138ms/step - loss: 0.2432 - acc: 0.5828 - val_loss: 0.2475 - val_acc: 0.5902
Epoch 5/30
734/734 [==============================] - 102s 139ms/step - loss: 0.2449 - acc: 0.5849 - val_loss: 0.2419 - val_acc: 0.5902
Epoch 6/30
734/734 [==============================] - 101s 138ms/step - loss: 0.2434 - acc: 0.5804 - val_loss: 0.2522 - val_acc: 0.4098
Epoch 7/30
734/734 [==============================] - 101s 137ms/step - loss: 0.2446 - acc: 0.5832 - val_loss: 0.2436 - val_acc: 0.5902
Epoch 8/30
734/734 [============================

KeyboardInterrupt: 

## Result visualisation 

In [6]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='lower left')
plt.title('Training and Validation Loss')

plt.show()

NameError: name 'history' is not defined